# Improvement Trial

Previously I forgot to manage memory properly, but this time I will also look at the score each player got in each game

In [1]:
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 1.3

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []



TEMPERATURE: 1.000
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: -1.1
NUMBER OF TURNS AVERAGE: 381.1
---PLAYER 0 STATS---


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [3]:
for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/225.)
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])

---PLAYER 0 STATS---
WIN AVERAGE: 0.236
SCORE AVERAGE: -24.8
---PLAYER 1 STATS---
WIN AVERAGE: 0.236
SCORE AVERAGE: -25.1
---PLAYER 2 STATS---
WIN AVERAGE: 0.173
SCORE AVERAGE: -28.2
---PLAYER 3 STATS---
WIN AVERAGE: 0.187
SCORE AVERAGE: -26.0
---PLAYER 4 STATS---
WIN AVERAGE: 0.182
SCORE AVERAGE: -25.4


In [4]:
temperatures = [0.5, 0.2, 0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.0001, 0., 0.]

for temperature in temperatures:
    print 'TEMPERATURE: %.03f' % temperature
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(3):
        print 'in cycle %d' % cycle
        for i in range(75):
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config)
            game.run()
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/225.)
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

TEMPERATURE: 0.500
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: -1.9
NUMBER OF TURNS AVERAGE: 376.7
---PLAYER 0 STATS---
WIN AVERAGE: 0.293
SCORE AVERAGE: -22.6
---PLAYER 1 STATS---
WIN AVERAGE: 0.178
SCORE AVERAGE: -29.0
---PLAYER 2 STATS---
WIN AVERAGE: 0.191
SCORE AVERAGE: -25.9
---PLAYER 3 STATS---
WIN AVERAGE: 0.196
SCORE AVERAGE: -27.0
---PLAYER 4 STATS---
WIN AVERAGE: 0.164
SCORE AVERAGE: -29.1
TEMPERATURE: 0.200
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: -2.7
NUMBER OF TURNS AVERAGE: 378.7
---PLAYER 0 STATS---
WIN AVERAGE: 0.422
SCORE AVERAGE: -17.4
---PLAYER 1 STATS---
WIN AVERAGE: 0.129
SCORE AVERAGE: -30.0
---PLAYER 2 STATS---
WIN AVERAGE: 0.138
SCORE AVERAGE: -29.1
---PLAYER 3 STATS---
WIN AVERAGE: 0.173
SCORE AVERAGE: -27.8
---PLAYER 4 STATS---
WIN AVERAGE: 0.147
SCORE AVERAGE: -28.7
TEMPERATURE: 0.100
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: 1.8
NUMBER OF TURNS AVERAGE: 496.5
---PLAYER 0 STATS---
WIN AVERAGE: 0.462
SCORE AVERAGE: -13

KeyboardInterrupt: 

$a^2$

In [ ]:
print game.mt
print game.tl
print game.rt

In [ ]:
game.tickets

In [ ]:
np.mean(overall_winning_scores,1)

In [ ]:
np.mean(overall_player_scores, ( 2,))

In [ ]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test004_%%s.h5' % i)

not sure why all player scores improved for the last row...perhaps training should lower temperature more quickly